### Libraries

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

### Import

In [65]:
df_1 = pd.read_csv(r"C:\Users\ashis\Documents\Cricket\CS_IPL_ALL_WITH_BAT_POS.csv")

# df_1.drop('Unnamed: 0', axis=1, inplace=True, )

df_1.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'batting_pos'],
      dtype='object')

In [66]:
(df_1.season.unique())

array(['2007/08', '2009', '2009/10', '2011', '2012', 2012, 2013, 2014,
       2015, 2016, 2017, 2019, 2018, '2018', '2019', '2020/21', '2021',
       2021, 2022, 2023], dtype=object)

In [67]:
df_1.loc[df_1.season=='2007/08', 'season'] = 2008
df_1.loc[df_1.season=='2009/10', 'season'] = 2010
df_1.loc[df_1.season=='2020/21', 'season'] = 2020

df_1['season'] = df_1['season'].astype('int')

# Merges

In [68]:
# Runs per season for every player for every season
season_runs  = df_1.groupby(['season', 'striker']).agg({'runs_off_bat':'sum', 'ball':'count'}).reset_index()

# Accounting Wide Balls
extras2 = df_1[(df_1['wides']>0)].groupby(['season', 'striker']).agg({'ball':'count'}).reset_index()

# Merging Wide Balls
merged = pd.merge(season_runs, extras2, on=['season', 'striker'], how='outer').fillna(0)

# Subtracting Wide Balls from total Balls to get Balls played by batters
merged['balls'] = merged["ball_x"] - merged["ball_y"]

# Dropping useless columns
merged.drop(['ball_x', 'ball_y'], axis=1, inplace=True)

# Generating SR
merged['SR'] = round((merged['runs_off_bat']/merged['balls'])*100, 2)

# Getting innings count for batters by season
inns = df_1.groupby(['season', 'striker']).agg({'match_id':pd.Series.nunique}).reset_index()

# Merging innings count with first merge
merged2 = pd.merge(merged, inns, on=['season', 'striker'], how='outer').rename({'match_id':'inns'}, axis=1)

# Getting OUTS count of every batter for every season
outs = pd.DataFrame(df_1.groupby('season')['player_dismissed'].value_counts()).rename({'player_dismissed':'Outs'}, axis=1).reset_index()

# Renaming outs
outs.rename({'player_dismissed':'striker'}, axis=1, inplace=True)

# Merging outs to previous merge
merged3 = pd.merge(merged2, outs, on=['season', 'striker'], how='outer')

In [69]:
#Cleaning Merges
merged3.fillna(0, inplace=True)

merged3.sort_values(['season', 'runs_off_bat'],inplace=True)

# Creating BpD
merged3['BpD'] = round(merged3['balls']/merged3['Outs'], 2)

In [70]:
merged3.head(2)

,season,striker,runs_off_bat,balls,SR,inns,Outs,BpD
3,2008,A Mukund,0.0,1.0,0.0,1.0,1.0,1.0
15,2008,Abdur Razzak,0.0,2.0,0.0,1.0,0.0,inf


# Creating the same for Venue level

In [71]:
match_by_match_individual_inns = df_1.fillna(0)[df_1.fillna(0)['wides']==0].groupby(['season', 'match_id', 'start_date', 'venue' ,'striker']).agg({'runs_off_bat':'sum', 'ball':'count'}).reset_index()

In [72]:
match_by_match_individual_inns.head(2)

,season,match_id,start_date,venue,striker,runs_off_bat,ball
0,2008,335982,2008-04-18,M Chinnaswamy Stadium,AA Noffke,9,10
1,2008,335982,2008-04-18,M Chinnaswamy Stadium,B Akhil,0,2


# Top 25 runs scorer of all time

In [73]:
df_top_all = df_1.groupby('striker').agg({'runs_off_bat':'sum'}).reset_index().sort_values('runs_off_bat', ascending=False)

df_top_all.reset_index(inplace=True)

In [74]:
df_top_all.drop('index', axis=1, inplace=True)

In [75]:
top_25_scorers = np.array(df_top_all.nlargest(25, 'runs_off_bat').striker)

In [76]:
top_25_scorers

array(['V Kohli', 'S Dhawan', 'DA Warner', 'RG Sharma', 'SK Raina',
       'AB de Villiers', 'MS Dhoni', 'CH Gayle', 'RV Uthappa',
       'KD Karthik', 'AM Rahane', 'AT Rayudu', 'G Gambhir', 'KL Rahul',
       'F du Plessis', 'SV Samson', 'SR Watson', 'MK Pandey',
       'KA Pollard', 'SA Yadav', 'JC Buttler', 'YK Pathan', 'Q de Kock',
       'BB McCullum', 'RR Pant'], dtype=object)

# Number of seasons

In [77]:
all_seasons = match_by_match_individual_inns.season.unique()

In [86]:
all_seasons

array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,
       2019, 2020, 2021, 2022, 2023], dtype=int64)

# Venue Analysis functions

In [79]:
def venue_analysis(df, player, year):
    df_dict = {}
    
    inns = df[(df['season']==year) & (df['striker']==player)]

    player_venue = inns.groupby('venue').agg({'runs_off_bat':'sum', 'ball':'sum', 'match_id':'count'}).reset_index().sort_values('runs_off_bat', ascending=False)

    player_venue['RpO'] =  round((player_venue['runs_off_bat']/player_venue['ball'])*6, 4)
    
    venue_stats = df[df['season']==year].groupby('venue').agg({'runs_off_bat':'sum', 'ball':'sum'}).reset_index()
    
    venue_stats['RpO_Venue'] = round((venue_stats['runs_off_bat']/venue_stats['ball'])*6, 4)

    v_venue = pd.merge(player_venue, venue_stats[['venue', 'RpO_Venue']], on='venue')

    v_venue['Run Diff.'] = v_venue['RpO'] - v_venue['RpO_Venue']

    v_venue['Weighted Matches'] = v_venue['match_id']/v_venue['match_id'].sum()

    v_venue['Weighted RpO'] = v_venue['Run Diff.']*v_venue['Weighted Matches']
    
    return (v_venue['Weighted RpO'].sum()/v_venue['match_id'].sum())*10
    #return v_venue
    #return v_venue['Weighted RpO'].sum()

In [80]:
# NVRR analysis for all the top 25 scorers
dict_NVRR = {}

for player in top_25_scorers:
    l1 = []
    #for year in seasons_for_player(match_by_match_individual_inns, player):
    for year in all_seasons:
        NVRR = round(venue_analysis(match_by_match_individual_inns, player, year), 3)
        l1.append((year,NVRR))
    dict_NVRR[player]=l1

In [81]:
# Creating a DF with the dictinary
df_seasons_NVRR = pd.DataFrame(dict_NVRR).T

In [82]:
df_seasons_NVRR.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
V Kohli,"(2008, -1.012)","(2009, -0.182)","(2010, 0.178)","(2011, -0.137)","(2012, -0.908)","(2013, 0.176)","(2014, -0.672)","(2015, -0.171)","(2016, 0.297)","(2017, -0.924)","(2018, -0.502)","(2019, -0.338)","(2020, -0.403)","(2021, -0.202)","(2022, -0.601)","(2023, -0.719)"
S Dhawan,"(2008, -1.655)","(2009, -5.707)","(2010, -1.786)","(2011, 0.004)","(2012, 0.378)","(2013, -0.294)","(2014, -0.718)","(2015, -0.706)","(2016, -0.563)","(2017, -0.169)","(2018, -0.444)","(2019, -0.229)","(2020, 0.512)","(2021, -0.191)","(2022, -0.5)","(2023, -0.294)"


In [87]:
df_seasons_NVRR_v1 = df_seasons_NVRR.rename({k:v for (k, v) in zip(list(range(16)), all_seasons)}, axis=1)

In [88]:
df_seasons_NVRR_v1.head(2)

,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
V Kohli,"(2008, -1.012)","(2009, -0.182)","(2010, 0.178)","(2011, -0.137)","(2012, -0.908)","(2013, 0.176)","(2014, -0.672)","(2015, -0.171)","(2016, 0.297)","(2017, -0.924)","(2018, -0.502)","(2019, -0.338)","(2020, -0.403)","(2021, -0.202)","(2022, -0.601)","(2023, -0.719)"
S Dhawan,"(2008, -1.655)","(2009, -5.707)","(2010, -1.786)","(2011, 0.004)","(2012, 0.378)","(2013, -0.294)","(2014, -0.718)","(2015, -0.706)","(2016, -0.563)","(2017, -0.169)","(2018, -0.444)","(2019, -0.229)","(2020, 0.512)","(2021, -0.191)","(2022, -0.5)","(2023, -0.294)"


In [89]:
# Getting NVMM out of (year, NVMM) format

df_seasons_NVRR_v1[2008] = df_seasons_NVRR_v1[2008].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

df_seasons_NVRR_v1[2009] = df_seasons_NVRR_v1[2009].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

df_seasons_NVRR_v1[2010] = df_seasons_NVRR_v1[2010].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

df_seasons_NVRR_v1[2011] = df_seasons_NVRR_v1[2011].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

df_seasons_NVRR_v1[2012] = df_seasons_NVRR_v1[2012].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

df_seasons_NVRR_v1[2013] = df_seasons_NVRR_v1[2013].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

df_seasons_NVRR_v1[2014] = df_seasons_NVRR_v1[2014].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

df_seasons_NVRR_v1[2015] = df_seasons_NVRR_v1[2015].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

df_seasons_NVRR_v1[2016] = df_seasons_NVRR_v1[2016].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

df_seasons_NVRR_v1[2017] = df_seasons_NVRR_v1[2017].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

df_seasons_NVRR_v1[2018] = df_seasons_NVRR_v1[2018].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

df_seasons_NVRR_v1[2019] = df_seasons_NVRR_v1[2019].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

df_seasons_NVRR_v1[2020] = df_seasons_NVRR_v1[2020].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

df_seasons_NVRR_v1[2021] = df_seasons_NVRR_v1[2021].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

df_seasons_NVRR_v1[2022] = df_seasons_NVRR_v1[2022].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

df_seasons_NVRR_v1[2023] = df_seasons_NVRR_v1[2023].astype('str').str.split(',', expand=True).iloc[:, 1].str.split(')', expand=True).iloc[:, 0]

In [90]:
df_seasons_NVRR_v1.head(2)

,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
V Kohli,-1.012,-0.182,0.178,-0.137,-0.908,0.176,-0.672,-0.171,0.297,-0.924,-0.502,-0.338,-0.403,-0.202,-0.601,-0.719
S Dhawan,-1.655,-5.707,-1.786,0.004,0.378,-0.294,-0.718,-0.706,-0.563,-0.169,-0.444,-0.229,0.512,-0.191,-0.5,-0.294


In [91]:
df_seasons_NVRR_v2 = df_seasons_NVRR_v1.replace(' nan', 0)

In [92]:
df_seasons_NVRR_v2.head(2)

,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
V Kohli,-1.012,-0.182,0.178,-0.137,-0.908,0.176,-0.672,-0.171,0.297,-0.924,-0.502,-0.338,-0.403,-0.202,-0.601,-0.719
S Dhawan,-1.655,-5.707,-1.786,0.004,0.378,-0.294,-0.718,-0.706,-0.563,-0.169,-0.444,-0.229,0.512,-0.191,-0.5,-0.294


In [93]:
# Convert all to numeric float values
df_seasons_NVRR_v2 = df_seasons_NVRR_v2.apply(pd.to_numeric)

In [94]:
# Initial Heatmap

# df_seasons_NVRR_v2.style.background_gradient(cmap ='hot')

In [95]:
# Re-indexing
df_seasons_NVRR_v3 = df_seasons_NVRR_v2.reset_index()

# Getting players column
df_seasons_NVRR_v3.rename({'index':'Player'}, inplace=True, axis=1)

In [96]:
df_seasons_NVRR_v3.head(2)

,Player,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,V Kohli,-1.012,-0.182,0.178,-0.137,-0.908,0.176,-0.672,-0.171,0.297,-0.924,-0.502,-0.338,-0.403,-0.202,-0.601,-0.719
1,S Dhawan,-1.655,-5.707,-1.786,0.004,0.378,-0.294,-0.718,-0.706,-0.563,-0.169,-0.444,-0.229,0.512,-0.191,-0.500,-0.294


In [28]:
#Seaborn Heatmaps on v2 df

# import seaborn as sns

# sns.set_style('darkgrid')
# sns.set(rc={'figure.figsize':(14,8)})

# ax = sns.lineplot(data=df_seasons_NVRR_v3, x ='Player', y = df_seasons_NVRR_v3.columns,
#                   hue='District', palette='viridis',
#                   legend='full', lw=3)

# ax.xaxis.set_major_locator(ticker.MultipleLocator(4))
# plt.legend(bbox_to_anchor=(1, 1))
# plt.ylabel('PM2.5 (µg/m3)')
# plt.xlabel('Year-Month')
# plt.show()

# sns.heatmap(df_seasons_NVRR_v2, annot=True, cmap='hot')
# plt.savefig('NVRR_heatmap.png')
# plt.show()

In [97]:
# Filling wides value coz in places where there are no wides, it is NULL by deafult so filling no wides with 0

df_1.wides = df_1.wides.fillna(0)

# Non wides run agg for total runs in the IPL till 2023 start
df_1[df_1['wides']==0].groupby('striker').agg({'runs_off_bat':'sum', 'ball':'count','match_id':pd.Series.nunique}).reset_index().sort_values('runs_off_bat', ascending=False).head(5)

,striker,runs_off_bat,ball,match_id
597,V Kohli,7273,5590,229
488,S Dhawan,6617,5205,216
138,DA Warner,6399,4577,176
455,RG Sharma,6213,4779,237
519,SK Raina,5536,4046,200


In [99]:
# 350+ runs seasons info

seasons_needed = merged3[merged3['striker'].isin(top_25_scorers)][merged3['runs_off_bat']>200]

seasons_needed.shape

(256, 8)

In [101]:
# Melting to Player, season, NVRR value

melted = pd.melt(df_seasons_NVRR_v3, id_vars=['Player'])

melted.rename({'variable':'season', 'value':'NVRR'}, axis=1, inplace=True)

melted.head(5)

,Player,season,NVRR
0,V Kohli,2008,-1.012
1,S Dhawan,2008,-1.655
2,DA Warner,2008,0.000
3,RG Sharma,2008,-0.002
4,SK Raina,2008,0.474


In [102]:
# Renaming merged3 to join later

merged3.rename({'striker':'Player'}, axis=1, inplace=True)

In [103]:
# Merging melted DF with merged3 to get a df with NVRR and other values together

merged_top25 = pd.merge(melted, merged3, on=['Player', 'season'], how='left')

final_df_v1 = merged_top25[merged_top25['runs_off_bat']>350][['Player', 'season', 'runs_off_bat', 'BpD', 'NVRR', 'SR']]

In [104]:
final_df_v1.shape

(156, 6)

In [105]:
final_df_v1['Name'] = list(zip(final_df_v1.Player, final_df_v1.season))

In [106]:
final_df_v1.head(5)

,Player,season,runs_off_bat,BpD,NVRR,SR,Name
3,RG Sharma,2008,404.0,24.82,-0.002,147.99,"(RG Sharma, 2008)"
4,SK Raina,2008,421.0,26.82,0.474,142.71,"(SK Raina, 2008)"
6,MS Dhoni,2008,414.0,31.00,-0.331,133.55,"(MS Dhoni, 2008)"
12,G Gambhir,2008,534.0,29.15,-0.201,140.90,"(G Gambhir, 2008)"
16,SR Watson,2008,472.0,31.10,0.858,151.77,"(SR Watson, 2008)"


In [108]:
fig = px.scatter(final_df_v1, x = 'BpD', y='NVRR', text='Player')
fig.update_traces(textposition='top center')
fig.add_hline(y=0)
fig.add_vline(x=20)
# #fig.update_layout(title_text='Life Expectency', title_x=0.5)
fig.show()

In [40]:
# final_df_v1.to_csv("350runs_CuttOff.csv")

In [109]:
# No filter
merged_top25[['Player', 'season', 'runs_off_bat', 'BpD', 'NVRR', 'SR']].shape

(400, 6)

In [110]:
# 200 runs filter
merged_top25[merged_top25['runs_off_bat']>200][['Player', 'season', 'runs_off_bat', 'BpD', 'NVRR', 'SR']].shape

(256, 6)

In [111]:
# 350 runs filter
merged_top25[merged_top25['runs_off_bat']>350][['Player', 'season', 'runs_off_bat', 'BpD', 'NVRR', 'SR']].shape

(156, 6)

In [48]:
# merged3.to_csv("All_Player_IPL_Seasons_Till2022.csv")

In [47]:
# top_25_scorers

In [51]:
# match_by_match_individual_inns.to_csv('match_by_match_individual_inns_till_2022.csv')

In [112]:
final_df_v1[final_df_v1['Player']=='V Kohli']

,Player,season,runs_off_bat,BpD,NVRR,SR,Name
75,V Kohli,2011,557.0,38.33,-0.137,121.09,"(V Kohli, 2011)"
100,V Kohli,2012,364.0,25.08,-0.908,111.66,"(V Kohli, 2012)"
125,V Kohli,2013,639.0,32.79,0.176,139.22,"(V Kohli, 2013)"
150,V Kohli,2014,359.0,22.62,-0.672,122.11,"(V Kohli, 2014)"
175,V Kohli,2015,505.0,35.09,-0.171,130.83,"(V Kohli, 2015)"
200,V Kohli,2016,973.0,53.33,0.297,152.03,"(V Kohli, 2016)"
250,V Kohli,2018,530.0,34.64,-0.502,139.11,"(V Kohli, 2018)"
275,V Kohli,2019,464.0,23.43,-0.338,141.46,"(V Kohli, 2019)"
300,V Kohli,2020,471.0,35.09,-0.403,122.02,"(V Kohli, 2020)"
325,V Kohli,2021,405.0,24.21,-0.202,119.47,"(V Kohli, 2021)"
